# Exercise 10.1: "Calculation of Pi"

1. Extend/revise your code from earlier on estimating Pi using Monte Carlo simulation to use an OpenCL kernel for parallel GPU processing.

In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# A short template to test small kernels.
#

import numpy as np
import pyopencl as cl

DIM = 1024
N = 10*24*DIM

# Create the host side data and a empty array to hold the result.
x_host = np.random.rand(N).astype(np.float64)
y_host = np.random.rand(N).astype(np.float64)
result_host = np.zeros(N//DIM).astype(np.int32)

# Create the context (containing platform and device information) and command queue.
context = cl.create_some_context()
cmd_queue = cl.CommandQueue(context)

# Create a device side read-only memory buffer and copy the data from "hostbuf" into it.
# Create as many
# You can find the other possible mem_flags values at
# https://www.khronos.org/registry/OpenCL/sdk/1.2/docs/man/xhtml/clCreateBuffer.html
mf = cl.mem_flags
x_device = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=x_host)
y_device = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=y_host)
result_device = cl.Buffer(context, mf.WRITE_ONLY, result_host.nbytes)

# Source of the kernel itself.
kernel_source = f"""#define DIM {DIM}""" + """
__kernel void in_circle(
    __global const double *rndx_device, 
    __global const double *rndy_device, 
    __global       int    *result_device)
{
  __local int ldata_device[DIM];
  int lid = get_local_id(0);
  int gid = get_global_id(0);
  int groupId = get_group_id(0);

  double x = rndx_device[gid];
  double y = rndy_device[gid];
  ldata_device[lid] = (int)((x*x + y*y) <= 1);
  barrier(CLK_LOCAL_MEM_FENCE);
  if(lid == 0)
  {
    int count = 0;
    for(int i = 0; i < DIM; i++)
    {
      count += ldata_device[i];

    }
    result_device[groupId] = count;
  }
}
"""

# If you want to keep the kernel in a seperate file uncomment this line and adjust the filename
#kernel_source = open("kernel.cl").read()

# Create a new program from the kernel and build the source.
prog = cl.Program(context, kernel_source).build()

# Execute the "sum" kernel in the program. Parameters are:
#
#        Command queue         Work group size   Kernel param 1
#            ↓   Global grid size   ↓   Kernel param 0  ↓  Kernel param 2
#            ↓           ↓          ↓       ↓           ↓        ↓
prog.in_circle(cmd_queue, x_host.shape, (DIM,),
               x_device, y_device, result_device)

# Copy the result back from device to host.
cl.enqueue_copy(cmd_queue, result_host, result_device)

pi_estimate = 4*sum(result_host)/N

print(pi_estimate)


3.1423665364583333


2. Extend your code to make a reduction in each work group, i.e. compute the avg. estimate of Pi in each work group. The result array should only contain one value per work group.